# Calligaro Nicolas
## Affichage d'une carte et des données
Readme :\
Ce jupyter se concentre sur l'affichage du carte et d'élément dessus.
Runner l'ensemble du jupyter

In [1]:
import pandas as pd
import ipyleaflet
from ipyleaflet import basemaps,Map,Marker,AntPath,FullScreenControl, Popup, Polyline,LayersControl
import ipywidgets as widgets
import pickle
import seaborn as sns
df_clas=pickle.Unpickler(open("df_clas.pkl",'rb')).load()
df_tech=pickle.Unpickler(open("df_tech.pkl",'rb')).load()
df_tech.shape,df_clas.shape

((33, 22), (4142, 23))

# Création d'un ant_path sur une carte

In [3]:
def Ant_Path_Skipper(list_skipper):
    """
    On produit des ant_path : chemin animé
    pour chaque skipper passé dans la liste
    attention : fait vite ramer
    """
    path = {}
    for i in list_skipper:#df['N°'].unique(): #on n'en fait qu'un seul pour test
        pos_skip= []
        for x,y in zip(list(df_clas.loc[df_clas['Num_voile'] == i]['Latitude']),list(df_clas.loc[df_clas['Num_voile'] == i]['Longitude'])):
            if (x and y) :#protection contre les False si donnée manquante
                pos_skip.append([x,y])
        path[i] = AntPath(
                         locations=[
                             pos_skip
                         ]
                        ,use = 'polyline' #‘polyline’, ‘polygon’, ‘rectangle’ and ‘circle’ 
                        ,radius= 10 #Radius of the circle, if use is set to ‘circle’
                        ,weight =5 #Epaisseur du trait
                        ,dash_array=[10, 100] #nb de fourmi sur le chemin
                        ,delay=1000 #vitesse des fourmis
                        ,color='#7590ba' #couleur de fond
                        ,pulse_color='#3f6fba' #couleur de la fourmi
                        ,name=df_tech.loc[i].Skipper
                        )
    return path

In [4]:
def polyline_skipper(list_skipper):
    """
    On produit des polyline, comme ant_path mais non animé
    """
    df_clas.Skipper
    pal = sns.color_palette('rainbow', 20)
    pal2= sns.color_palette("Spectral", as_cmap=True)
    line={}
    for i in list_skipper:
        pos_skip= []
        for x,y in zip(list(df_clas.loc[df_clas['Num_voile'] == i]['Latitude']),list(df_clas.loc[df_clas['Num_voile'] == i]['Longitude'])):
            if (x and y) :#protection contre les False si données manquantes
                pos_skip.append([x,y])
                
        line[i] = Polyline(locations=[pos_skip]
                           ,color= pal.as_hex()[i%20] #Couleur
                           ,fill=False
                           ,stroke = True #Bordure
                           ,opacity=1.0
                           ,weight=1 #largeur en pixel
                           #,fill_color = None #couleur du remplissage
                           #,fill_opacity = 0.2 #opacité du remplissage
                           #,dash_array = None
                           #,line_cap = 'round'
                           #,line_join = 'round'
                           ,name=df_tech.loc[i].Skipper
                          )
    return line

In [5]:
import seaborn as sns
pal = sns.color_palette('rainbow', 15)
pal.as_hex()[10]

'#e0d377'

In [5]:
#On calcul tous les trajet en Ant_path et en polyline
top3 = list(df_clas.sort_values(by=['Date full','Rang'],ascending=[False,True])['Num_voile'].head(3))
last10 = list(df_clas.sort_values(by=['Date full','Rang'],ascending=[False,True])['Num_voile'])[3:13]
other = list(df_clas[~df_clas.Num_voile.isin(top3)]['Num_voile'].unique())
full = list(df_clas.Num_voile.unique())
antpath = Ant_Path_Skipper(top3)
polyline = polyline_skipper(last10)

In [6]:
#On affiche les 3 premiers en ant_path.
#Le reste en polyline.

m = Map(center=(46, -1), zoom=2, basemap = basemaps.Stamen.Watercolor)

#Le ant path affice les 3 premiers dans l'ordre
for key in antpath.keys():
    m.add_layer(antpath[key])
    
#le polyline n'a pas d'ordre
for key in polyline.keys():
    m.add_layer(polyline[key])  

m.add_control(LayersControl(position='topright'))

m.add_control(FullScreenControl())

display(m)

Map(center=[46, -1], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…